In [2]:
from sentence_transformers import SentenceTransformer
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

c:\Users\humming\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


c:\Users\humming\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [458]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
import pandas as pd 
import numpy as np
from numpy import dot
from numpy.linalg import norm

# EdgeDriver 경로 설정
edge_driver_path = 'C:/Users/humming/Downloads/GPT2_Chatbot/msedgedriver.exe'

# Service 객체 생성
service = Service(edge_driver_path)
options = webdriver.EdgeOptions()
options.add_argument("headless")
# EdgeDriver 인스턴스 생성
driver = webdriver.Edge(service=service, options=options)

url = 'https://ko.wikipedia.org/wiki/%EB%82%98%EB%9D%BC_%EC%9D%B4%EB%A6%84%EC%88%9C_%EC%88%98%EB%8F%84_%EB%AA%A9%EB%A1%9D'
# 웹페이지 로딩
url2 = 'https://ko.wikipedia.org/wiki/%EC%A1%B0%EC%84%A0'
driver.get(url)

# 웹 요소 조작
element = driver.find_element(By.XPATH,"/html/body/div[2]/div/div[3]/main/div[3]/div[3]/div[1]")
list = element.text.split('\n')
driver.get(url2)
ttt = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[3]/main/div[3]/div[3]/div[1]/table[1]/tbody')
korea= ttt.text
nation_list = []
for x  in list:
    if '-' in x:
        nation_list.append(x.split(' - '))

nationdf = pd.DataFrame(nation_list, columns=['국가', '수도'])
nationdf['수도'][20]='서울특별시'

de = pd.read_csv('____________.csv', encoding='cp949')
for  i in range(len(nationdf)):
    if pd.notnull(de['다른명칭'][i]):
        nationdf['국가'].iloc[i]=[nationdf['국가'].iloc[i]]+de['다른명칭'][i].split(', ')
    else:
        nationdf['국가'].iloc[i]=[nationdf['국가'].iloc[i]]
nationdf['embedding'] = nationdf.apply(lambda row: model.encode([row.국가[::-1]]) if len(row.국가)>1 else model.encode(row.국가) , axis = 1)

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))
def return_answer(question):
    embedding = model.encode(question)
    nationdf['score'] = nationdf.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    return nationdf.loc[nationdf['score'].idxmax()]['수도']

C:\Users\humming\AppData\Local\Temp\ipykernel_43256\4202249719.py:36: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  nationdf['수도'][20]='서울특별시'
C:\Users\humming\AppData\Local\Temp\ipykernel_43256\4202249719.py:43: FutureWarning: ChainedAssign

In [456]:
nationdf['국가'][150]

['조선민주주의인민공화국', '조선민주주의인민공화국(북한)', '북한']

In [462]:
return_answer('우즈베키스탄의 수도는?')

'타슈켄트'

In [461]:
return_answer('조선민주주의인민공화국의 수도는?')

'평양'

In [444]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from datasets import load_dataset
import os


os.environ['KMP_DUPLICATE_LIB_OK']='True'
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
# KorQuAD 데이터셋 로드
dataset = load_dataset('KorQuAD/squad_kor_v2', trust_remote_code=True)


contexts = dataset['train']['context']
questions = dataset['train']['question']

# 각 문맥에 대한 임베딩 계산
context_embeddings = [model.encode(context) for context in tqdm(contexts, desc="Context Embedding")]


# 코사인 유사도를 통해 질문에 맞는 문맥을 찾는 함수
def answer_question(input_question, contexts, context_embeddings):
    # 입력된 질문에 대한 임베딩 계산
    question_embedding = model.encode(input_question)
    
    # 각 문맥과의 코사인 유사도 계산
    similarities = [cosine_similarity([question_embedding], [context_embedding]).flatten()[0] for context_embedding in context_embeddings]
    
    # 유사도가 가장 높은 문맥 선택
    best_match_idx = similarities.index(max(similarities))
    best_context = contexts[best_match_idx]
    best_similarity = similarities[best_match_idx]
    
    return best_context, best_similarity



Loading dataset shards:   0%|          | 0/29 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
input_question = "세종대왕은 무엇을 했나요?"

# 질문에 대한 답변 찾기
answer, similarity = answer_question(input_question, contexts, context_embeddings)

print(f"질문: {input_question}")
print(f"답변: {answer}")
print(f"유사도: {similarity}")